In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RelMap,
    SubMap,
    SelIndex,
    DB
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RelMap(
            rel=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RelMap(
                    rel=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RelMap(
                            rel=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RelMap(
                    rel=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    link=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [7]:
with cProfile.Profile() as pr:
  db = DB()
  rec = await data_source.load([data], db=db)
  
  stats = pstats.Stats(pr)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\base.py:1967 in       │
│ _exec_single_context                                                                             │
│                                                                                                  │
│   1964 │   │   │   │   │   │   │   evt_handled = True                                            │
│   1965 │   │   │   │   │   │   │   break                                                         │
│   1966 │   │   │   │   if not evt_handled:                                                       │
│ ❱ 1967 │   │   │   │   │   self.dialect.do_execute(                                              │
│   1968 │   │   │   │   │   │   cursor, str_statement, effective_parameters, context              │
│   1969 │   │   │   │   │   )                                                                     │
│   1970                                                                                           │
│                                                                                                  │
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\sqlalchemy\engine\default.py:941 in     │
│ do_execute                                                                                       │
│                                                                                                  │
│    938 │   │   cursor.executemany(statement, parameters)                                         │
│    939 │                                                                                         │
│    940 │   def do_execute(self, cursor, statement, parameters, context=None):                    │
│ ❱  941 │   │   cursor.execute(statement, parameters)                                             │
│    942 │                                                                                         │
│    943 │   def do_execute_no_params(self, cursor, statement, context=None):                      │
│    944 │   │   cursor.execute(statement)                                                         │
│                                                                                                  │
│ c:\Users\lworm\Repos\py-research\.venv\Lib\site-packages\duckdb_engine\__init__.py:140 in        │
│ execute                                                                                          │
│                                                                                                  │
│   137 │   │   │   elif parameters is None:                                                       │
│   138 │   │   │   │   self.__c.execute(statement)                                                │
│   139 │   │   │   else:                                                                          │
│ ❱ 140 │   │   │   │   self.__c.execute(statement, parameters)                                    │
│   141 │   │   except RuntimeError as e:                                                          │
│   142 │   │   │   if e.args[0].startswith("Not implemented Error"):                              │
│   143 │   │   │   │   raise NotImplementedError(*e.args) from e                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ConstraintException: Constraint Error: NOT NULL constraint failed: test_db_schema_Project.org__id

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 with cProfile.Profile() as pr:                                                          

In [8]:
[s for s in db[Search]]

[{'term': None, 'result_count': 3}]

In [9]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         100327 function calls (95753 primitive calls) in 0.345 seconds

   Ordered by: cumulative time
   List reduced from 2037 to 94 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.028    0.028 C:\Users\lworm\Repos\py-research\src\py_research\db\base.py:4351(engine)
        2    0.000    0.000    0.004    0.002 C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:234(full_map)
        2    0.000    0.000    0.004    0.002 C:\Users\lworm\Repos\py-research\src\py_research\db\data_source.py:350(_push_to_pull_map)
   430/36    0.000    0.000    0.003    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:54(inner)
   430/36    0.001    0.000    0.003    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:69(gen_int_hash)
       68    0.000    0.000    0.003    0.000 C:\Users\lworm\Repos\py-research\src\py_research\reflect\types.py:61(has_type)
        6 